In [23]:
import re
import sys
import pprint
import json
from IPython.display import display
from colorama import Fore, Back, Style
import subprocess

In [308]:
def scan():
    global token_index, nxtsymb, cursymb
    token_index += 1
    if token_index < len(tokens_chain):
        cursymb = tokens_chain[token_index-1]
        nxtsymb = tokens_chain[token_index]

In [245]:
def antiscan():
    global token_index, nxtsymb, cursymb
    token_index -= 1
    if token_index >= 1:
        cursymb = tokens_chain[token_index-1]
        nxtsymb = tokens_chain[token_index]

In [334]:
def error(text):
    global mistakes
    if text[-8:] == 'expected':
        if text == '";" expected' and (cursymb[3] != nxtsymb[3]):
            mistakes.append(str(text) + ' in the end of line ' + str(cursymb[3]))
            return
        text += ' instead of "' + nxtsymb[2] + '"'
    mistakes.append(str(text) + ' at line ' + str(nxtsymb[3]))

In [247]:
def check(response):
    if response:
        error(response)
        return False
    else:
        return True

In [322]:
def wrapper():
    scan()
    if nxtsymb[2] == 'program':
        if not check(program()):
            return 'invalid program declaration'
        
        scan()
    if nxtsymb[2] == 'var':
        scan()
        if not check(var()):
            return 'invalid var declaration'
        scan()
        if nxtsymb[2] != ';':
            return '";" expected'
        scan()
        while nxtsymb[0] == 'I':
            if not check(var()):
                return 'invalid var declaration'
            scan()
            if nxtsymb[2] != ';':
                return '";" expected'
            scan()
    
    while nxtsymb[2] in ['function', 'procedure']:
        if not check(function_or_procedure()):
            return 'invalid function or procedure declaration'
    
    if nxtsymb[2] != 'begin':
        return '"begin" expected'
    scan()
    
    mistake = ''
    while not mistake:
        mistake = operator()
        if mistake:
            return mistake
        if cursymb[0] == 'I' and nxtsymb[2] == ':':
            scan()
        else:
            scan()
            if nxtsymb[2] == ';':
                scan()
                if nxtsymb[2] == 'end.':
                    break
            else:
                return '";" expected' 

In [269]:
def program():
#     print(Fore.GREEN + 'program' + Style.RESET_ALL)
    scan()
    if nxtsymb[0] != 'I':
        return 'identifier expected'
    scan()
    if nxtsymb[2] != ';':
        return '";" expected'
    return ''

In [270]:
def var():
#     print(Fore.GREEN + 'var' + Style.RESET_ALL)
    if nxtsymb[0] != 'I':
        return 'identifier expexted'
    scan()
    while nxtsymb[2] == ',':
        scan()
        if nxtsymb[0] != 'I':
            return 'identifier expected'
        scan()
    if nxtsymb[2] != ':':
        return '":" expected'
    scan()
    if not check(var_type()):
        return 'invalid variable type declaration'
    return ''

In [271]:
def var_type():
#     print(Fore.GREEN + 'var_type' + Style.RESET_ALL)
    if nxtsymb[2] in ['integer', 'real', 'string']:
        return ''
    elif nxtsymb[2] == 'array':
        scan()
        if nxtsymb[2] != '[':
            return '"[" expected'
        scan()
        if not check(interval()):
            return 'invalid interval syntax'
        scan()
        while nxtsymb[2] == ',':
            scan()
            if not check(interval()):
                return 'invalid interval syntax'
            scan()
        if nxtsymb[2] != ']':
            return '"]" expected'
        scan()
        if nxtsymb[2] != 'of':
            return '"of" expected'
        scan()
        if nxtsymb[2] not in ['integer', 'real', 'string']:
            return 'variable type expected'
        return ''
    else:
        return 'array type expected'
        
            

In [272]:
def interval():
#     print(Fore.GREEN + 'interval' + Style.RESET_ALL)
    if nxtsymb[0] != 'C':
        return 'constant expected'
    scan()
    if nxtsymb[2] != '..':
        return '".." expected'
    scan()
    if nxtsymb[0] != 'C':
        return 'constant expected'
    return ''

In [273]:
def function_or_procedure():
#     print(Fore.GREEN + 'function_or_procedure' + Style.RESET_ALL)
    if nxtsymb[2] == 'procedure':
        scan()
        if not check(procedure()):
            return 'invalid procedure declaration'
        return ''
    if nxtsymb[2] == 'function':
        scan()
        if not check(function()):
            return 'invalid function declaration'
        return ''
    return ''        

In [315]:
def procedure():
#     print(Fore.GREEN + 'procedure' + Style.RESET_ALL)
    if nxtsymb[0] != 'I':
        return 'identifier expended'
    scan()
    if nxtsymb[2] == '(':
        scan()
        if not check(var()):
            return 'invalid var declaration'
        scan()
        while nxtsymb[2] == ';':
            scan()
            if not check(var()):
                return 'invalid var declaration'
            scan()
        if nxtsymb[2] != ')':
            return '")" expected'
        scan()
    if nxtsymb[2] != ';':
        return '";" expected'
    
    scan()
    if nxtsymb[2] == 'var':
        scan()
        if not check(var()):
            return 'invalid var declaration'
        scan()
        if nxtsymb[2] != ';':
            return '";" expected'
        scan()
        while nxtsymb[0] == 'I':
            if not check(var()):
                return 'invalid var declaration'
            scan()
            if nxtsymb[2] != ';':
                return '";" expected'
            scan()
    
    while nxtsymb[2] in ['function', 'procedure']:
        if not check(function_or_procedure()):
            return 'invalid function or procedure declaration'
    
    if nxtsymb[2] != 'begin':
        return '"begin" expected'
    scan()
    mistake = ''
    while not mistake:
        if nxtsymb[2] == 'end':
            break
        mistake = operator()
        if mistake:
            return mistake
        if cursymb[0] == 'I' and nxtsymb[2] == ':':
            scan()
        else:
            scan()
            if nxtsymb[2] == ';':
                scan()
            else:
                return '";" expected'
            
    if nxtsymb[2] != 'end':
        if cursymb[2] == ';' and nxtsymb[2] == 'else':
            return 'extra ";" before else'
        return 'unexpected "'+nxtsymb[2]+'"'
    scan()
    if nxtsymb[2] != ';':
        return '";" expected'
    scan()
    return ''

In [314]:
def function():
#     print(Fore.GREEN + 'function' + Style.RESET_ALL)
    if nxtsymb[0] != 'I':
        return 'identifier expended'
    scan()
    if nxtsymb[2] == '(':
        scan()
        if not check(var()):
            return 'invalid var declaration'
        scan()
        while nxtsymb[2] == ';':
            scan()
            if not check(var()):
                return 'invalid var declaration'
            scan()
        if nxtsymb[2] != ')':
            return '")" expected'
        scan()
    if nxtsymb[2] != ':':
        return 'expected ":"'
    scan()
    if nxtsymb[2] not in ['integer', 'real', 'string']:
        return 'type of function expected'
    scan()
    if nxtsymb[2] != ';':
        return '";" expected'
    
    scan()
    if nxtsymb[2] == 'var':
        scan()
        if not check(var()):
            return 'invalid var declaration'
        scan()
        if nxtsymb[2] != ';':
            return '";" expected'
        scan()
        while nxtsymb[0] == 'I':
            if not check(var()):
                return 'invalid var declaration'
            scan()
            if nxtsymb[2] != ';':
                return '";" expected'
            scan()
    
    while nxtsymb[2] in ['function', 'procedure']:
        if not check(function_or_procedure()):
            return 'invalid function or procedure declaration'
    
#     scan()
    if nxtsymb[2] != 'begin':
        return '"begin" expected'
    
    scan()
    return_presence = False
    mistake = ''
    while not mistake:
        if nxtsymb[2] == 'return':
            return_presence = True
        if nxtsymb[2] == 'end':
            break
        mistake = operator(True)
        if mistake:
            return mistake
        if cursymb[0] == 'I' and nxtsymb[2] == ':':
            scan()
        else:
            scan()
            if nxtsymb[2] == ';':
                scan()
            else:
                return '";" expected'

    if not return_presence:
        return 'return expected in the end of function'
            
    if nxtsymb[2] != 'end':
        if cursymb[2] == ';' and nxtsymb[2] == 'else':
            return 'extra ";" before else'
        return 'unexpected "'+nxtsymb[2]+'"'
    scan()
    if nxtsymb[2] != ';':
        return '";" expected'
    scan()
    return ''

In [276]:
def operator(and_return = False):
#     print(Fore.GREEN + 'operator' + Style.RESET_ALL)
    if nxtsymb[2] == ';':
        pass
    elif nxtsymb[0] == 'I':
        scan()
        if nxtsymb[2] == ':':
            return ''
        if nxtsymb[2] == '[':
            scan()
            if not check(expression()):
                return 'invalid array arguments'
            scan()
            while nxtsymb[2] == ',':
                scan()
                if not check(expression()):
                    return 'invalid array arguments'
                scan()
            if nxtsymb[2] != ']':
                return '"]" expected'
            scan()
        if nxtsymb[2] == ':=':
            if not check(assignment()):
                return 'invalid assignment declaration'
            return ''
        if nxtsymb[2] == '(':
            scan()
            if not check(expression()):
                return 'invalid argument'
            scan()
            while nxtsymb[2] == ',':
                scan()
                if not check(expression()):
                    return 'invalid argument'
                scan()
            if nxtsymb[2] != ')':
                return 'expected ")"'
            return ''
        return 'unexpected "'+nxtsymb[2]+'"'
    elif nxtsymb[2] == 'if':
        scan()
        if not check(condition()):
            return 'condition expected'
        scan()
        if nxtsymb[2] != 'then':
            return '"then" expected'
        scan()
        if not check(operator()):
            return 'operator expected'
        scan()
        if nxtsymb[2] == 'else':
            scan()
            if not check(operator()):
                return 'operator expected'
            return ''
        elif nxtsymb[2] == ';':
            scan()
            if nxtsymb[2] == 'else':
                antiscan()
                return 'extra ";" before "else"'
            antiscan()
        antiscan()
        return ''
    elif nxtsymb[2] == 'goto':
        scan()
        if nxtsymb[0] != 'I':
            return 'identifier expected'
        return ''
    elif nxtsymb[2] == 'while':
        scan()
        if not check(condition()):
            return 'condition expected'
        scan()
        if nxtsymb[2] != 'do':
            return '"do" expected'
        scan()
        if not check(operator()):
            return 'operator expected'
    elif and_return and nxtsymb[2] == 'return':
        scan()
        if not check(expression()):
            return 'return\'s exression expected'
        return ''
    else:
        return 'operator expected'
    return ''

In [277]:
def condition():
#     print(Fore.GREEN + 'condition' + Style.RESET_ALL)    
    if not check(expression()):
        return 'expression expected'
    scan()
    if nxtsymb[2] not in ['=', '>', '<', '>=', '<=', '<>']:
        return 'comparasion operation'
    scan()
    if not check(expression()):
        return 'expression expected'
    return ''

In [278]:
def assignment():
#     print(Fore.GREEN + 'assignment' + Style.RESET_ALL)
    if nxtsymb[2] != ':=':
        return '":=" expected'
    scan()
    if not check(expression()):
        return 'invalid expression'
    return ''

In [279]:
def expression():
#     print(Fore.GREEN + 'expression' + Style.RESET_ALL)    
    if not check(term()):
        return 'term expected'
    scan()
    while nxtsymb[2] in ['+', '-']:
        scan()
        if not check(term()):
            return 'term expected'
        scan()
    else:
        antiscan()
        pass
        
    return ''

In [300]:
def term():
#     print(Fore.GREEN + 'term' + Style.RESET_ALL)
    if not check(factor()):
        return 'factor expected'
    scan()
    while nxtsymb[2] in ['*', '/', '^']:
        scan()
        if not check(factor()):
            return 'factor expected'
        scan()
    else:
        antiscan()
        pass
        
    return ''

In [281]:
def factor():
#     print(Fore.GREEN + 'factor' + Style.RESET_ALL)
    if nxtsymb[2] == '(':
        scan()
        if not check(expression()):
            return 'expression expected'
        scan()
        if nxtsymb[2] != ')':
            return '")" expected'
    else:
        if not check(argument()):
            return 'constant or identifier expected'
    return ''

In [282]:
def argument():
#     print(Fore.GREEN + 'argument' + Style.RESET_ALL)
    if nxtsymb[0] not in ['I', 'C']:
        return 'constant or identifier expected'
    scan()
    if nxtsymb[2] in ['[', '(']:
        scan()
        if not(check(expression())):
            return 'invalid expression'
        scan()
        while nxtsymb[2] == ',':
            scan()
            if not(check(expression())):
                return 'invalid expression'
            scan()
        if nxtsymb[2] not in [']', ')']:
            return 'expected \']\' or \')\''
    else:
        antiscan()
    return ''

In [337]:
subprocess.run(["python3", "lexical-analyzer.py"])
with open('./results/lab1.json') as lab1_file:
    data = json.load(lab1_file)

tokens_chain = data['chain']
service_words = data['tables']['service_words']
operations = data['tables']['operations']
separators = data['tables']['separators']
constants = data['tables']['constants']
identifiers = data['tables']['identifiers']

nxtsymb = []
cursymb = []
token_index = -1

mistakes = []

def is_mistakes():
    check(wrapper())
    if mistakes:
        return mistakes[0]
    return False
is_mistakes()

'";" expected in the end of line 24'